In [1]:
import pandas as pd

In [2]:
train=pd.read_csv(r"C:\Users\shubham\Desktop\Untitled Folder\airline_sentiment_analysis.csv")


In [3]:
train.head()

,Unnamed: 0,airline_sentiment,text
0,1,positive,@VirginAmerica plus you've added commercials t...
1,3,negative,@VirginAmerica it's really aggressive to blast...
2,4,negative,@VirginAmerica and it's a really big bad thing...
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [4]:
train.shape

(11541, 3)

In [5]:
#data cleaning
def preprocessing_text(table):
    #put everything in lowercase
    table['text'] = table['text'].str.lower()
    #Replace links contained in the tweet
    table['text'] = table['text'].replace(r'((www\.[^\s]+)|(https?://[^\s]+))', 'URL', regex=True)
    #remove numbers
    table['text'] = table['text'].replace(r'[0-9]+', '', regex=True)
    #replace special characters and puntuation marks
    table['text'] = table['text'].replace(r'[!"$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
    table['text'] = table['text'].replace(r'#([^\s]+)', r'\1', regex=True)
    return table

In [6]:
#remove stop words
def stop_words(table):
    #We need to remove the stop words
    stop_words_list = stopwords.words('english')
    table['text'] = table['text'].str.lower()
    table['text'] = table['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words_list)]))
    return table

In [7]:
def cleaning_table(table):
    #This function will process all the required cleaning for the text in our tweets
    table = preprocessing_text(table)

    table = stop_words(table)
    return table

In [8]:
#check missing value
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11541 entries, 0 to 11540
Data columns (total 3 columns):
Unnamed: 0           11541 non-null int64
airline_sentiment    11541 non-null object
text                 11541 non-null object
dtypes: int64(1), object(2)
memory usage: 270.6+ KB


In [9]:
#drop the missing values
train.dropna(inplace=True)

In [10]:
import re
import nltk
from nltk.corpus import stopwords

In [11]:
train = cleaning_table(train)

In [12]:
train.head()

,Unnamed: 0,airline_sentiment,text
0,1,positive,virginamerica plus added commercials experienc...
1,3,negative,virginamerica really aggressive blast obnoxiou...
2,4,negative,virginamerica really big bad thing
3,5,negative,virginamerica seriously would pay flight seats...
4,6,positive,virginamerica yes nearly every time fly vx “ea...


In [13]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import (wordnet,stopwords)

In [14]:
from nltk.stem.porter import *
train['tokenized_tweet'] = [nltk.word_tokenize(x) for x in train['text']]
stemmer = PorterStemmer()
train['tokenized_tweet'] = train['tokenized_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
train['tokenized_tweet'] = train['tokenized_tweet'].apply(lambda x: ' '.join(x))
train.head()

,Unnamed: 0,airline_sentiment,text,tokenized_tweet
0,1,positive,virginamerica plus added commercials experienc...,virginamerica plu ad commerci experi tacki
1,3,negative,virginamerica really aggressive blast obnoxiou...,virginamerica realli aggress blast obnoxi ente...
2,4,negative,virginamerica really big bad thing,virginamerica realli big bad thing
3,5,negative,virginamerica seriously would pay flight seats...,virginamerica serious would pay flight seat pl...
4,6,positive,virginamerica yes nearly every time fly vx “ea...,virginamerica ye nearli everi time fli vx “ ea...


In [15]:
X = train['text']
y = train['airline_sentiment']

In [22]:
y=y.map({'positive':1,'negative':0})

In [18]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
X= vectorizer.fit_transform(X).toarray()

In [20]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
y

0        1
1        0
2        0
3        0
4        1
        ..
11536    0
11537    0
11538    1
11539    0
11540    0
Name: airline_sentiment, Length: 11541, dtype: int64

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [27]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=77)

In [28]:
lr = LogisticRegression(random_state=77, class_weight='balanced')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

0.9023967658099913
0.9044896654286048
              precision    recall  f1-score   support

           0       0.95      0.92      0.94      2722
           1       0.74      0.84      0.79       741

    accuracy                           0.90      3463
   macro avg       0.85      0.88      0.86      3463
weighted avg       0.91      0.90      0.90      3463

